# DOWNLOAD CATS v DOGS DATASET AND STORE IN HDF5 FORMAT

In [ ]:
!git clone https://github.com/vinayakshenoy/DL4CV
%cd DL4CV/dogs_vs_cats

Cloning into 'DL4CV'...
remote: Enumerating objects: 191, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 191 (delta 68), reused 93 (delta 28), pack-reused 0
Receiving objects: 100% (191/191), 3.27 MiB | 2.99 MiB/s, done.
Resolving deltas: 100% (68/68), done.
/content/DL4CV/dogs_vs_cats


In [ ]:
!kaggle competitions download -c dogs-vs-cats
!echo "{\"username\":\"vinayakshenoy\",\"key\":\"d1b8f531b7524b3a963c1b5df2846af3\"}" >> /root/.kaggle/kaggle.json
!mkdir -p datasets/hdf5
!mkdir output
%cd datasets
!kaggle competitions download -c dogs-vs-cats
!unzip train.zip
!rm train.zip
!rm test1.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/cat.11551.jpg     
  inflating: train/cat.11552.jpg     
  inflating: train/cat.11553.jpg     
  inflating: train/cat.11554.jpg     
  inflating: train/cat.11555.jpg     
  inflating: train/cat.11556.jpg     
  inflating: train/cat.11557.jpg     
  inflating: train/cat.11558.jpg     
  inflating: train/cat.11559.jpg     
  inflating: train/cat.1156.jpg      
  inflating: train/cat.11560.jpg     
  inflating: train/cat.11561.jpg     
  inflating: train/cat.11562.jpg     
  inflating: train/cat.11563.jpg     
  inflating: train/cat.11564.jpg     
  inflating: train/cat.11565.jpg     
  inflating: train/cat.11566.jpg     
  inflating: train/cat.11567.jpg     
  inflating: train/cat.11568.jpg     
  inflating: train/cat.11569.jpg     
  inflating: train/cat.1157.jpg      
  inflating: train/cat.11570.jpg     
  inflating: train/cat.11571.jpg     
  inflating: train/cat.11572.jpg     
  inflating: train/cat.11573.jpg     

In [ ]:
%cd /content/DL4CV/dogs_vs_cats/

[Errno 2] No such file or directory: '/content/DL4CV/dogs_vs_cats/'
/content


In [ ]:
!pip install import_ipynb
!pip install h5py
!pip install tqdm

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=fa5897eb512d663d89bf8914cbbcb19f60945045339e951c31b389755f7118ce
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


## MAIN SCRIPT TO STORE IN HDF5

In [ ]:
!python build_dogs_vs_cats.py

importing Jupyter notebook from ../pyimage/preprocessing/preprocessors.ipynb
2020-08-21 02:54:38.255857: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
importing Jupyter notebook from ../pyimage/io/hdf5py.ipynb
[INFO] building datasets/hdf5/train.hdf5...
Building Dataset: 100% |########################################| Time:  0:12:43
[INFO] building datasets/hdf5/val.hdf5...
Building Dataset: 100% |########################################| Time:  0:01:21
[INFO] building datasets/hdf5/test.hdf5...
Building Dataset: 100% |########################################| Time:  0:01:24
[INFO] serializing means...


# NOTEBOOK CODE TO TRAIN
- Generate data in batches from hdf5 train file
  - Include augmentation using aug.flow
  - Include preprocessing
- Build model and train

In [ ]:
# import the necessary packages
import sys
sys.path.append("../")
import import_ipynb
import matplotlib.pyplot as plt
from config import dogs_vs_cats_config as config
from pyimage.preprocessing.preprocessors import ImageToArrayPreprocessor, SimplePreprocessor,PatchPreprocessor, MeanPreprocessor
from pyimage.callbacks.training_monitor import TrainingMonitor
from pyimage.io.hdf5py import HDF5DatasetGenerator, HDF5DatasetWriter
from pyimage.nn.alexnet import AlexNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import json
import os

importing Jupyter notebook from ../pyimage/preprocessing/preprocessors.ipynb
importing Jupyter notebook from ../pyimage/callbacks/training_monitor.ipynb
importing Jupyter notebook from ../pyimage/io/hdf5py.ipynb
importing Jupyter notebook from ../pyimage/nn/alexnet.ipynb


In [ ]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, 
                         width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
                         horizontal_flip=True, fill_mode="nearest")

In [ ]:
# load the RGB means for training set
means = json.loads(open(config.DATASET_MEAN).read())

# init image preprocessors

sp = SimplePreprocessor(227, 227)
pp = PatchPreprocessor(227, 227)
mp = MeanPreprocessor(means["R"], means["G"], means["B"])
iap = ImageToArrayPreprocessor()

In [ ]:
# initialize the training and validation dataset generators
trainGen = HDF5DatasetGenerator(config.TRAIN_HDF5, 128, aug=aug,
                                preprocessors=[pp, mp, iap], classes=2)
valGen = HDF5DatasetGenerator(config.VAL_HDF5, 128,
                              preprocessors=[sp, mp, iap], classes=2)

In [ ]:
# initialize the optimizer
print("[INFO] compiling model...")
opt = Adam(lr=1e-3)
model = AlexNet.build(width=227, height=227, depth=3, 
                      classes=2, reg=0.0002)
model.compile(loss="binary_crossentropy", optimizer=opt, 
              metrics=["accuracy"])

[INFO] compiling model...


In [ ]:
# construct the set of callbacks
path = os.path.sep.join([config.OUTPUT_PATH, "{}.png".format(
os.getpid())])
callbacks = [TrainingMonitor(path)]

In [ ]:
H = model.fit_generator(
    trainGen.generator(), 
    steps_per_epoch=trainGen.numImages // 128,
    validation_data=valGen.generator(), 
    validation_steps=valGen.numImages // 128,
    epochs=75,
    max_queue_size=10, 
    verbose=1)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/75
156/156 [==============================] - 1226s 8s/step - loss: 4.0550 - accuracy: 0.5417 - val_loss: 4.3774 - val_accuracy: 0.5222
Epoch 2/75
156/156 [==============================] - 1223s 8s/step - loss: 2.7697 - accuracy: 0.5718 - val_loss: 2.4193 - val_accuracy: 0.5892
Epoch 3/75
156/156 [==============================] - 1219s 8s/step - loss: 2.2843 - accuracy: 0.5686 - val_loss: 2.3828 - val_accuracy: 0.5452
Epoch 4/75
156/156 [==============================] - 1222s 8s/step - loss: 1.9029 - accuracy: 0.5829 - val_loss: 2.0228 - val_accuracy: 0.5666
Epoch 5/75
156/156 [==============================] - 1223s 8s/step - loss: 1.6495 - accuracy: 0.5993 - val_loss: 1.6671 - val_accuracy: 0.5938
Epoch 6/75
156/156 [==============================] - 1223s 8s/step - loss: 1.4000 - accuracy: 0.6336 - val_loss: 1.2385 - val_accuracy: 0.6756
Epoch 7/75
156/156 [==============================] - 1222s 

In [ ]:
# save the model to file
print("[INFO] serializing model...")
model.save(config.MODEL_PATH, overwrite=True)

# close the HDF5 datasets
trainGen.close()
valGen.close()

# Evaluating AlexNet: Cats v Dogs
- crop_accuracy.py

In [ ]:
# The HDF5DatasetGenerator is required so we can access the testing set of
# our dataset and obtain predictions on this data using our pre-trained model.

from config import dogs_vs_cats_config as config
from pyimage.preprocessing.preprocessors import CropPreprocessor
from pyimage.utils.ranked import rank5_accuracy
from tensorflow.kerass.models import load_model

In [ ]:
# load the RGB means for training set
means = json.loads(open(config.DATASET_MEAN).read())

# init the image preprocessors
sp = SimplePreprocessor(227,227) # resizes image to 227x227
mp = MeanPreprocessor(means["R"], means["G"], means["B"])
cp = CropPreprocessor(227, 227) # takes random 227x227 path from image
iap = ImageToArrayPreprocessor()

print("INFO loading model...")
model = load_model(config.MODEL_PATH)

In [ ]:
# Before we apply over-sampling and 10-cropping, let’s first obtain a baseline on the testing set
# using only the original testing image as input to our network:

# init testing dataset generator, then make predictions on test data
print("INFO predicting on test data (no crops")
testGen = HDF5Generator(config.TEST_HDF5, 64, preprocessors=[sp, mp, iap], classes=2)
predictions = model.predict_generator(testGen.generator(),
                                      steps=testGen.numImages//64, max_queue_size=10)

# rank1 and rank5 accuracies
(rank1, _) = rank5_accuracy(predictions, testGen.db["labelss"])
testGen.close()

In [ ]:
# re init the testing set generator, this time excluding the 'SimplePreprocessor'
testGen = HDF5DatasetGenerator(config.TEST_HDF5, 64, preprocessors=[mp], classes=2)
predictions = []

# init progress bar
widgets = ["Evaluating: ", progressbar.Percentage(), " ",progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=testGen.numImages//64,
                               widgets=widgets).start()

In [ ]:
# loop over a single pass of test data
for (i, (images, labels)) in enumerate(testGen.generator(passes=1)):
  # loop over each of the individual images
  for image in images:
    # apply the crop preprocessor to the image to generate 10
    # separate crops, then convert them from images to arrays
    crops = cp.preprocess(image)
    crops = np.array([iap.preprocess(c) for c in crops], dtype="float32")
    
    #make predictions on crops and avg them to obtain final predictions
    pred = model.predict(crops)
    predictions.append(pred.mean(axis=0))

  pbar.update(i)

In [ ]:
# compute the rank-1 accuracy
pbar.finish()
print("[INFO] predicting on test data (with crops)...")
(rank1, _) = rank5_accuracy(predictions, testGen.db["labels"])
print("[INFO] rank-1: {:.2f}%".format(rank1 * 100))
testGen.close()

# Transfer learning on Cats v dogs

## Extracting Features from ResNet50

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
import import_ipynb
import sys
sys.path.append("../")
from imutils import paths
import numpy as np
import progressbar
import random
import os

In [ ]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [ ]:
from tqdm.notebook import trange, tqdm

In [ ]:
args = {
    "dataset":"datasets/train/",
    "output":"datasets/hdf5/features.hdf5",
    "batch_size":16,
    "buffer_size":1000
}

bs = args["batch_size"]

In [ ]:
# grab the list of images that we'll be describing then randomly
# shuffle them to allow for easy training and testing splits via 
# array slicing during training time
print("INFO loading images")
imagePaths = list(paths.list_images(args["dataset"]))
random.shuffle(imagePaths)

# extract class labels from image paths then encode the labels
labels = [p.split(os.path.sep)[-1].split(".")[0] for p in imagePaths]
le = LabelEncoder()
labels = le.fit_transform(labels)

INFO loading images


In [ ]:
# load ResNet50 network
print("INFO loading network...")
model = ResNet50(weights="imagenet", include_top=False)
model.summary()

INFO loading network...
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
___________________________________________________________________

In [ ]:
!rm datasets/hdf5/features.hdf5

In [ ]:
# init HDF5 dataset writer then store the class label names in dataset
dataset = HDF5DatasetWriter((len(imagePaths), 100352),
                            args["output"], dataKey="features", bufSize=args["buffer_size"])
dataset.storeClassLabels(le.classes_)

In [ ]:
# initialize the progress bar
widgets = ["Extracting Features: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=len(imagePaths), widgets=widgets).start()

Extracting Features: N/A% |                                    | ETA:  --:--:--

In [ ]:
# with tqdm(total=len(imagePaths)) as pbar:
numBatches = len(imagePaths)//bs
batch_no = 0
for i in np.arange(0, len(imagePaths), bs):
  # extract the batch of images and labels, then init the
  # list of actual images that will be passed through network
  # for feature extraction
  batchPaths = imagePaths[i:i+bs]
  batchLabels = labels[i:i+bs]
  batchImages = []

  # loop over images and labels in current batch
  for (j, imagePath) in enumerate(batchPaths):
    # load input image using keras helper utility
    # while ensuring image resized to 224x224
    image = load_img(imagePath, target_size=(224,224))
    image = img_to_array(image)

    # preprocess the image by expanding the dimensions 
    # and subtracting the mean RGB of imagenet dataset
    image = np.expand_dims(image, axis=0)
    image = imagenet_utils.preprocess_input(image)

    # add image to batch
    batchImages.append(image)
  
  # pass batch of images through network to extract features
  batchImages = np.vstack(batchImages)
  features = model.predict(batchImages, batch_size=bs)

  # reshape features so that each image is represented by 
  # flatten feature vector
  features = features.reshape((features.shape[0], 100352))

  # add features and labels to HDF5 dataset
  dataset.add(features, batchLabels)
  batch_no += 1
  if batch_no%50==0:
    print("Batches completed: {}/{}".format(batch_no, numBatches))
  #pbar.update(i)

# close dataset
dataset.close()
#pbar.finish()

Batches completed: 50/1562
Batches completed: 100/1562
Batches completed: 150/1562
Batches completed: 200/1562
Batches completed: 250/1562
Batches completed: 300/1562
Batches completed: 350/1562
Batches completed: 400/1562
Batches completed: 450/1562
Batches completed: 500/1562
Batches completed: 550/1562
Batches completed: 600/1562
Batches completed: 650/1562
Batches completed: 700/1562
Batches completed: 750/1562
Batches completed: 800/1562
Batches completed: 850/1562
Batches completed: 900/1562
Batches completed: 950/1562
Batches completed: 1000/1562
Batches completed: 1050/1562
Batches completed: 1100/1562
Batches completed: 1150/1562
Batches completed: 1200/1562
Batches completed: 1250/1562
Batches completed: 1300/1562
Batches completed: 1350/1562
Batches completed: 1400/1562
Batches completed: 1450/1562
Batches completed: 1500/1562
Batches completed: 1550/1562


## Training a Logistic Regression Classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import pickle
import h5py

In [ ]:
args = {
    "db":"datasets/hdf5/features.hdf5",
    "jobs":1
}

In [ ]:
# open the HDF5 database for reading then determine the index of
# the training and testing split, provided that this data was
# already shuffled *prior* to writing it to disk
db = h5py.File(args["db"], "r")
i = int(db["labels"].shape[0] * 0.75)

In [ ]:
# define the set of parameters that we want to tune then start a
# grid search where we evaluate our model for each value of C
params = {"C": [0.0001, 0.001, 0.01, 0.1, 1.0]}
model = GridSearchCV(LogisticRegression(solver="lbfgs", multi_class="auto"), params, cv=3, n_jobs=args["jobs"])
model.fit(db["features"][:i], db["labels"][:i])
print("INFO best hyperparameter: {}".format(model.best_params_))

In [ ]:
# generate a classification report for the model
print("[INFO] evaluating...")
preds = model.predict(db["features"][i:])
print(classification_report(db["labels"][i:], preds, target_names=db["label_names"])
                            
# compute raw acc.
acc = accuracy_score(db["labels"], preds)
print("INFO score: {}".format(acc))

In [ ]:
a = []
while(1):
  a.append('1')